In [1]:
%load_ext autoreload
%autoreload 2
import duckdb
import json
import os
import pandas as pd
import pathlib
import requests

### Integração com Reflora

In [2]:
family = "Piperaceae"
base_url = "https://servicos.jbrj.gov.br/v2/flora/species/%s" % family

In [3]:
response = requests.get(base_url)
if response.status_code == 200:
    j = response.json()
    if len(j) > 0:
        print("total de especies da %s eh: %d" % (family, len(j)))

total de especies da Piperaceae eh: 1042


In [4]:
if j:
    data = pd.read_json(json.dumps(j))
    data.to_parquet("reflora.parquet", index=False)

/tmp/ipykernel_446647/3777500992.py:2: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(json.dumps(j))


In [5]:
if os.path.exists("reflora.parquet"):
    conn = duckdb.connect("reflora.db")

    conn.execute("DROP TABLE IF EXISTS reflora")
    conn.execute(f"""
            CREATE TABLE reflora AS
            SELECT * FROM read_parquet(reflora.parquet, union_by_name=True)
        """)

    result = conn.execute("SELECT COUNT(*) FROM reflora").fetchone()
    print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 1,042


In [6]:
conn.execute("SELECT * FROM reflora").df()

taxonid                            scientificname taxonrank  \
0       20277                    Ottonia anisum Spreng.   ESPECIE   
1       12667               Peperomia loefgrenii Yunck.   ESPECIE   
2       12668                  Peperomia loxensis Kunth   ESPECIE   
3       12669            Peperomia lyman-smithii Yunck.   ESPECIE   
4       12670  Peperomia magnoliifolia (Jacq.) A.Dietr.   ESPECIE   
...       ...                                       ...       ...   
1037   637203         Piper emarginellum Sw. ex Wikstr.   ESPECIE   
1038   637204     Peperomia graveolens Rauh & Barthlott   ESPECIE   
1039   637205                Piper pereskiifolium Jacq.   ESPECIE   
1040   637206                         Piper alpinum Sw.   ESPECIE   
1041   637207                        Piper incanum Haw.   ESPECIE   

     nomenclaturalstatus taxonomicstatus  kingdom        phylum  \
0                   None        SINONIMO  Plantae  Tracheophyta   
1           NOME_CORRETO     NOME_ACEITO  Plantae  Tracheophyta   
2           NOME_CORRETO     NOME_ACEITO  Plantae  Tracheophyta   
3           NOME_CORRETO     NOME_ACEITO  Plantae  Tracheophyta   
4           NOME_CORRETO     NOME_ACEITO  Plantae  Tracheophyta   
...                  ...             ...      ...           ...   
1037                None        SINONIMO  Plantae  Tracheophyta   
1038                None     NOME_ACEITO  Plantae  Tracheophyta   
1039                None        SINONIMO  Plantae  Tracheophyta   
1040                None        SINONIMO  Plantae  Tracheophyta   
1041                None        SINONIMO  Plantae  Tracheophyta   

              class      order      family  ...  \
0     Magnoliopsida  Piperales  Piperaceae  ...   
1     Magnoliopsida  Piperales  Piperaceae  ...   
2     Magnoliopsida  Piperales  Piperaceae  ...   
3     Magnoliopsida  Piperales  Piperaceae  ...   
4     Magnoliopsida  Piperales  Piperaceae  ...   
...             ...        ...         ...  ...   
1037  Magnoliopsida  Piperales  Piperaceae  ...   
1038  Magnoliopsida  Piperales  Piperaceae  ...   
1039  Magnoliopsida  Piperales  Piperaceae  ...   
1040  Magnoliopsida  Piperales  Piperaceae  ...   
1041  Magnoliopsida  Piperales  Piperaceae  ...   

                                                 source  \
0     Guimarães, E.F.,Carvalho-Silva, M.,Monteiro, D...   
1     Guimarães, E.F.,Carvalho-Silva, M.,Monteiro, D...   
2     Guimarães, E.F.,Carvalho-Silva, M.,Monteiro, D...   
3     Guimarães, E.F.,Carvalho-Silva, M.,Monteiro, D...   
4     Guimarães, E.F.,Carvalho-Silva, M.,Monteiro, D...   
...                                                 ...   
1037  IPNI. https://www.ipni.org/n/681295-1\r\nTropi...   
1038  Lorenzi, H., Olsthoorn, G., Costa, C. Cactus e...   
1039   Tropicos. https://www.tropicos.org/name/25002991   
1040   Tropicos. https://www.tropicos.org/name/25002860   
1041  WFO. https://www.worldfloraonline.org/taxon/wf...   

            parentnameusage  parentnameusageid originalnameusageid  \
0           Ottonia Spreng.              12603             12746.0   
1     Peperomia Ruiz & Pav.              12609                 NaN   
2     Peperomia Ruiz & Pav.              12609                 NaN   
3     Peperomia Ruiz & Pav.              12609                 NaN   
4     Peperomia Ruiz & Pav.              12609            637209.0   
...                     ...                ...                 ...   
1037               Piper L.              12735                 NaN   
1038  Peperomia Ruiz & Pav.              12609                 NaN   
1039               Piper L.              12735             12688.0   
1040               Piper L.              12735            636123.0   
1041               Piper L.              12735             12665.0   

          namepublishedin namepublishedinyear  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3            

In [7]:
conn.execute("DESCRIBE reflora").df()

column_name   column_type null   key default extra
0                             taxonid        BIGINT  YES  None    None  None
1                      scientificname       VARCHAR  YES  None    None  None
2                           taxonrank       VARCHAR  YES  None    None  None
3                 nomenclaturalstatus       VARCHAR  YES  None    None  None
4                     taxonomicstatus       VARCHAR  YES  None    None  None
5                             kingdom       VARCHAR  YES  None    None  None
6                              phylum       VARCHAR  YES  None    None  None
7                               class       VARCHAR  YES  None    None  None
8                               order       VARCHAR  YES  None    None  None
9                              family       VARCHAR  YES  None    None  None
10                              genus       VARCHAR  YES  None    None  None
11                    specificepithet       VARCHAR  YES  None    None  None
12               infraspecificepithet        DOUBLE  YES  None    None  None
13           scientificnameauthorship       VARCHAR  YES  None    None  None
14                  acceptednameusage       VARCHAR  YES  None    None  None
15               higherclassification       VARCHAR  YES  None    None  None
16                             source       VARCHAR  YES  None    None  None
17                    parentnameusage       VARCHAR  YES  None    None  None
18                  parentnameusageid        BIGINT  YES  None    None  None
19                originalnameusageid        DOUBLE  YES  None    None  None
20                    namepublishedin       VARCHAR  YES  None    None  None
21                namepublishedinyear        DOUBLE  YES  None    None  None
22                         references       VARCHAR  YES  None    None  None
23                acceptednameusageid        DOUBLE  YES  None    None  None
24  bibliographiccitation_how_to_cite       VARCHAR  YES  None    None  None
25                           modified  TIMESTAMP_NS  YES  None    None  None

In [8]:
conn.execute("ALTER TABLE reflora ADD COLUMN is_endemic BOOL DEFAULT false")
conn.execute("SELECT is_endemic FROM reflora").df()

is_endemic
0          False
1          False
2          False
3          False
4          False
...          ...
1037       False
1038       False
1039       False
1040       False
1041       False

[1042 rows x 1 columns]

In [9]:
res = conn.execute("SELECT r.taxonid, r.references FROM reflora as r").df()
res

taxonid                                         references
0       20277  http://reflora.jbrj.gov.br/reflora/listaBrasil...
1       12667  http://reflora.jbrj.gov.br/reflora/listaBrasil...
2       12668  http://reflora.jbrj.gov.br/reflora/listaBrasil...
3       12669  http://reflora.jbrj.gov.br/reflora/listaBrasil...
4       12670  http://reflora.jbrj.gov.br/reflora/listaBrasil...
...       ...                                                ...
1037   637203  http://reflora.jbrj.gov.br/reflora/listaBrasil...
1038   637204  http://reflora.jbrj.gov.br/reflora/listaBrasil...
1039   637205  http://reflora.jbrj.gov.br/reflora/listaBrasil...
1040   637206  http://reflora.jbrj.gov.br/reflora/listaBrasil...
1041   637207  http://reflora.jbrj.gov.br/reflora/listaBrasil...

[1042 rows x 2 columns]

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

options = Options()
options.add_argument('--headless')  # Sem interface gráfica
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [27]:
import time

for idx, row in res.iterrows():
    print(idx, row["references"])
    
    try:
        driver.get(row["references"])        
        time.sleep(5)
        texto = driver.find_element(By.TAG_NAME, "body").text.lower()
        
        is_endemic = False
        for termo in ["endemic to brazil", "endêmica do brasil", "endêmico do brasil"]:
            if termo in texto:
                is_endemic = True
                break
        
        if is_endemic:
            print("%d eh endemico" % row["taxonid"])
            conn.execute(f"UPDATE reflora SET is_endemic = true WHERE taxonid = {row['taxonid']}")
            
    except Exception as e:
        print("erro no taxonid: %d" % row["taxonid"])
        continue

0 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB20277
1 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12667
12667 eh endemico
2 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12668
12668 eh endemico
3 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12669
12669 eh endemico
4 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12670
12670 eh endemico
5 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12671
12671 eh endemico
6 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12672
7 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC/FichaPublicaTaxonUC.do?id=FB12673
12673 eh endemico
8 http://reflora.jbrj.gov.br/reflora/listaBrasil/FichaPublicaTaxonUC

In [ ]:
conn.execute("SELECT count(*) FROM reflora as r where r.is_endemic").df()
driver.quit()

count_star()
0           477